In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import pywoe
import pandas as pd
import sklearn.datasets
import numpy

n_nas = 5
rng = numpy.random.default_rng(42)

iris = sklearn.datasets.load_iris()
X = pd.DataFrame(numpy.concatenate([iris.data, numpy.repeat(numpy.nan, 4 * n_nas).reshape(-1, iris.data.shape[1])], axis=0))
y = pd.Series(numpy.concatenate([(iris.target == 1).astype(int), numpy.array([rng.integers(0, 2) for _ in range(n_nas)])], axis=0))

categories = ['A', 'B', 'C', 'D', 'E', 'F']
probabilities = [
    [0.22, 0.2, 0.18, 0.16, 0.14, 0.1],
    [0.12, 0.16, 0.22, 0.22, 0.16, 0.12],
    [0.1, 0.14, 0.16, 0.18, 0.2, 0.22]
]

rng_cat = numpy.random.default_rng(42)
# X = numpy.concatenate([X, numpy.array([rng_cat.choice(categories, size=1, p=probabilities[target])[0] for target in y]).reshape(-1, 1)], axis=1)
X["categorical"] = numpy.array([rng_cat.choice(categories, size=1, p=probabilities[target])[0] for target in y])

class IrisData:
    X = X
    y = y

iris_data = IrisData()

In [11]:

woe = pywoe.PyWOE(min_leave_freq=0.05, random_state=42)
woe.fit(iris_data.X, iris_data.y)
display(woe.splits())
woe.woe_summary()

{'0': {'cuts': [-inf, 4.9, 5.2, 5.5, inf],
  'na_label': '[5.5, inf) OR NA',
  'labels': ['[-inf, 4.9)', '[4.9, 5.2)', '[5.2, 5.5)', '[5.5, inf) OR NA']},
 '1': {'cuts': [-inf, inf],
  'na_label': '[-inf, inf) OR NA',
  'labels': ['[-inf, inf) OR NA']},
 '2': {'cuts': [-inf, 1.9, 3.3, inf],
  'na_label': '[1.9, 3.3) OR NA',
  'labels': ['[-inf, 1.9)', '[1.9, 3.3) OR NA', '[3.3, inf)']},
 '3': {'cuts': [-inf, 0.5, inf],
  'na_label': '[0.5, inf) OR NA',
  'labels': ['[-inf, 0.5)', '[0.5, inf) OR NA']},
 'categorical': {'cuts': ['A', 'B', 'C', 'D', 'E', 'F'],
  'na_label': 'NA',
  'labels': ['A', 'B', 'C', 'D', 'E', 'F']}}

,variable,freq,events,nas,non_events,eventrate,woe,iv,group
0,0,16,0.5,0.0,16.5,0.031250,-2.831531,0.418737,"[-inf, 4.9)"
1,0,25,4.5,0.0,21.5,0.180000,-0.898999,0.109164,"[4.9, 5.2)"
2,0,11,2.5,0.0,9.5,0.227273,-0.670025,0.029602,"[5.2, 5.5)"
3,0,103,46.5,5.0,57.5,0.451456,0.452644,0.141900,"[5.5, inf) OR NA"
4,1,155,52.5,5.0,103.5,0.338710,0.000000,0.000000,"[-inf, inf) OR NA"
5,2,48,0.5,0.0,48.5,0.010417,-3.905206,1.775967,"[-inf, 1.9)"
6,2,8,3.5,5.0,5.5,0.437500,0.217520,0.002782,"[1.9, 3.3) OR NA"
7,2,99,49.5,0.0,50.5,0.500000,0.649505,0.287068,"[3.3, inf)"
8,3,48,0.5,0.0,48.5,0.010417,-3.900612,1.782237,"[-inf, 0.5)"
9,3,107,52.5,5.0,55.5,0.490654,0.618529,0.282614,"[0.5, inf) OR NA"


In [9]:
woe.woe_summary().to_dict(orient="list")

{'variable': ['0',
  '0',
  '0',
  '0',
  '1',
  '2',
  '2',
  '2',
  '3',
  '3',
  'categorical',
  'categorical',
  'categorical',
  'categorical',
  'categorical',
  'categorical'],
 'prediction': [0.3019354830826483,
  0.3179354835594854,
  0.32011730180632686,
  0.3465956785883831,
  0.33548387096774196,
  0.3019354830826483,
  0.33943548420023534,
  0.3514304340526622,
  0.3019354830826483,
  0.3505336153721069,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'freq': [16, 25, 11, 103, 155, 48, 8, 99, 48, 107, 29, 25, 38, 29, 22, 12],
 'events': [0.5,
  4.5,
  2.5,
  46.5,
  52.5,
  0.5,
  3.5,
  49.5,
  0.5,
  52.5,
  8.5,
  8.5,
  13.5,
  14.5,
  7.5,
  2.5],
 'minx': [4.3,
  4.9,
  5.2,
  5.5,
  2.0,
  1.0,
  1.9,
  3.3,
  0.1,
  0.5,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'maxx': [4.8,
  5.1,
  5.4,
  7.9,
  4.4,
  1.7,
  3.0,
  6.9,
  0.4,
  2.5,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'nas': [0.0,
  0.0,
  0.0,
  5.0,
  5.0,
  0.0,
  5.0,
  0.0,
  0.0,
  5.0,
  nan,
  n

In [4]:
X.head(10)

,0,1,2,3,categorical
0,5.1,3.5,1.4,0.2,E
1,4.9,3.0,1.4,0.2,C
2,4.7,3.2,1.3,0.2,E
3,4.6,3.1,1.5,0.2,D
4,5.0,3.6,1.4,0.2,A
5,5.4,3.9,1.7,0.4,F
6,4.6,3.4,1.4,0.3,E
7,5.0,3.4,1.5,0.2,E
8,4.4,2.9,1.4,0.2,A
9,4.9,3.1,1.5,0.1,C


In [5]:
# %%timeit
woe.transform(X).head(10)

,0,1,2,3,categorical
0,-0.898999,0.0,-3.905206,-3.900612,-0.069831
1,-2.831531,0.0,-3.905206,-3.900612,0.020117
2,-2.831531,0.0,-3.905206,-3.900612,-0.069831
3,-2.831531,0.0,-3.905206,-3.900612,0.589415
4,-0.898999,0.0,-3.905206,-3.900612,-0.271881
5,-0.670025,0.0,-3.905206,-3.900612,-0.778979
6,-2.831531,0.0,-3.905206,-3.900612,-0.069831
7,-0.898999,0.0,-3.905206,-3.900612,-0.069831
8,-2.831531,0.0,-3.905206,-3.900612,-0.271881
9,-2.831531,0.0,-3.905206,-3.900612,0.020117


In [6]:
%%timeit
woe.fit_transform(X, y)


149 ms ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
